In [ ]:
import pandas as pd
import sqlite3
import networkx as nx
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
import numpy as np

df = pd.read_csv("repeat_offenders.csv")

conn = sqlite3.connect("offenders.db")
cursor = conn.cursor()

cursor.execute("""
    CREATE TABLE IF NOT EXISTS complaints (
        Offender_ID TEXT,
        Name TEXT,
        Crime_Type TEXT,
        Date TEXT,
        Location TEXT,
        Complaint_Text TEXT,
        Case_Status TEXT
    )
""")

In [ ]:

df.to_sql("complaints", conn, if_exists="replace", index=False)

def get_repeat_offenders():
    query = "SELECT Name, COUNT(*) as Cases FROM complaints GROUP BY Name HAVING COUNT(*) > 1"
    return pd.read_sql(query, conn)

repeat_offenders = get_repeat_offenders()
print("\nRepeat Offenders:\n", repeat_offenders)

def get_crime_locations():
    query = "SELECT Location, Crime_Type, COUNT(*) as Cases FROM complaints GROUP BY Location, Crime_Type"
    return pd.read_sql(query, conn)

crime_locations = get_crime_locations()
print("\nCrime Locations:\n", crime_locations)


Repeat Offenders:
                  Name  Cases
0       Abigail Smith      3
1       Allison Smith      2
2      Amanda Jackson      3
3        Amanda Riley      2
4       Amanda Wilson      2
..                ...    ...
248  William Gonzalez      2
249   William Johnson      2
250    William Melton      2
251      William Reed      2
252   Zachary Johnson      2

[253 rows x 2 columns]

Crime Locations:
            Location     Crime_Type  Cases
0              None      Blackmail      1
1         Aaronland  Cyberstalking      1
2        Aaronmouth  Cyberstalking      1
3        Aaronmouth     Harassment      1
4        Aaronshire        Threats      1
...             ...            ...    ...
6301    Zacharyside     Harassment      1
6302      Zhangberg        Threats      1
6303     Zhangmouth     Harassment      1
6304  Zimmermanfurt       Stalking      1
6305     Zunigaland  Cyberstalking      1

[6306 rows x 3 columns]


In [ ]:
df["Latitude"] = np.random.uniform(-90, 90, len(df))
df["Longitude"] = np.random.uniform(-180, 180, len(df))

coords = np.array(df[["Latitude", "Longitude"]])
dbscan = DBSCAN(eps=10, min_samples=2).fit(coords)
df["Cluster"] = dbscan.labels_

print("\nClustering Results:\n", df[["Name", "Location", "Cluster"]])


def check_risk(offender, location):
    query = """
        SELECT COUNT(*) as Cases
        FROM complaints
        WHERE Name = ? AND Location = ?
    """
    result = cursor.execute(query, (offender, location)).fetchone()
    return result[0] > 1



Clustering Results:
                 Name         Location  Cluster
0     Travis Pearson       Brownburgh        0
1        James Lopez    Port Kimberly        0
2        Ryan Moreno        Port Cody        0
3      Kristi Graham        Ebonyview        0
4         Tyler Lutz      Brooksville        0
...              ...              ...      ...
6514    James Melton  Thompsonchester        0
6515      Tyler Cole     East Tiffany        0
6516     Billy Smith      Beltranview        0
6517    Joseph Berry     East Caitlyn        0
6518    Ryan Nichols              NaN        0

[6519 rows x 3 columns]


In [ ]:
offender_name = "Travis Pearson"
offender_location = "Brownburgh"
risk = check_risk(offender_name, offender_location)
print(f"\nRisk Assessment for {offender_name} at {offender_location}: {'High Risk' if risk else 'Low Risk'}")

conn.close()



Risk Assessment for Travis Pearson at Brownburgh: Low Risk
